In [2]:
import os
import sys
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Conv1D, Input, Dropout, MaxPooling1D, LSTM
from keras import optimizers
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

In [3]:
filehandler = open("../data/sample_data/hotel_reservation_only_bottlenecks_sample.pkl","rb")
dat = pickle.load(filehandler)
filehandler.close()
print(type(dat))
print(dat.shape)

<class 'pandas.core.frame.DataFrame'>
(3999996, 7)


In [4]:
batch_size = 4

In [5]:
df = dat.copy()
X = df.drop(columns = "label")
y = df.loc[:, "label"]

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = X_train.reset_index()
X_train = X_train.drop(columns = "index")
X_test = X_test.reset_index()
X_test = X_test.drop(columns = "index")

y_train = y_train.reset_index()
y_train = y_train.drop(columns = "index")
y_train = y_train.loc[:, "label"]
y_test = y_test.reset_index()
y_test = y_test.drop(columns = "index")
y_test = y_test.loc[:, "label"]


In [7]:
X_count = X_train.shape[0]
batch_count= X_count // batch_size

X_train_batch_list = []
y_train_batch_list = []


shuffled_range = range(X_count)
shuffled_X = X_train.iloc[shuffled_range, :]
shuffled_Y = [y_train[i] for i in shuffled_range]
for i in range(0, 10):

    y_train_batch = shuffled_Y[i*batch_size :(i +1)* batch_size]
    X_train_batch = shuffled_X[i*batch_size :(i +1)* batch_size]

    y_train_batch_list.append(y_train_batch)
    X_train_batch_list.append(X_train_batch)
        
        # vectorizer.fit_transform(train_data[i:i + batchSize])
        # Update the classifier with documents in the current mini-batch


In [8]:
train_data_pd = X_train_batch_list[0]
train_label = y_train_batch_list[0]
train_data = train_data_pd.to_numpy()

sample_size = train_data.shape[0] # number of samples in train set
time_steps  = train_data.shape[1] # number of features in train set
input_dimension = 1         

train_data_reshaped = train_data.reshape(sample_size,time_steps,input_dimension)
print("After reshape train data set shape:\n", train_data_reshaped.shape)
print("1 Sample shape:\n",train_data_reshaped[0].shape)
print("An example sample:\n", train_data_reshaped[0])


After reshape train data set shape:
 (4, 6, 1)
1 Sample shape:
 (6, 1)
An example sample:
 [[0.87874428]
 [0.02903283]
 [0.01747466]
 [0.02536331]
 [0.02607459]
 [0.02331032]]


In [10]:
def build_conv1D_model():
    n_timesteps = train_data_reshaped.shape[1] 
    n_features  = train_data_reshaped.shape[2] 
    model = None
    model = Sequential()
    model.add(Input(shape=(n_timesteps,n_features)))

#     model.add(Dense(64, activation = "sigmoid"))
#     model.add(Dense(64, activation = "sigmoid"))
#     model.add(Dense(64, activation = "sigmoid"))
#     model.add(Dense(64, activation = "sigmoid"))
#     model.add(Dense(64, activation = "sigmoid"))
#     model.add(Dense(64, activation = "sigmoid"))
    
    model.add(LSTM(4, activation = "tanh", return_sequences = True))
#     model.add(LSTM(64, activation = "tanh", return_sequences = True))
#     model.add(LSTM(64, activation = "tanh", return_sequences = True))
#     model.add(LSTM(64, activation = "tanh", return_sequences = True))
#     model.add(LSTM(64, activation = "tanh", return_sequences = True))
#     model.add(LSTM(64, activation = "tanh"))
    

#     model.add(Dense(64, activation = "sigmoid"))
#     model.add(Dense(64, activation = "sigmoid"))
#     model.add(Dense(64, activation = "sigmoid"))
#     model.add(Dense(64, activation = "sigmoid"))
#     model.add(Dense(64, activation = "sigmoid"))
#     model.add(Dense(64, activation = "sigmoid"))

    
#     model.add(Dense(6, activation = 'softmax'))

    optimizer = "adam"

    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

model_LSTM = build_conv1D_model()
model_LSTM.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 6, 4)              96        
                                                                 
Total params: 96
Trainable params: 96
Non-trainable params: 0
_________________________________________________________________


In [ ]:
X_train_arr = np.asarray(X_train.to_numpy())
y_train_arr = np.asarray(y_train.to_numpy())
history = model_LSTM.fit(X_train_arr, y_train_arr, epochs=10,
                            verbose=1, batch_size = 32)

In [1]:
acc = model_LSTM.evaluate(X_train, y_train)
print("Loss:", acc[0], " Accuracy:", acc[1])

NameError: name 'model_LSTM' is not defined

In [ ]:
pred = model_LSTM.predict(X_test)
pred_y = pred.argmax(axis=-1)
cm = confusion_matrix(y_test, pred_y)

In [ ]:
cm

In [ ]:
print(accuracy_score(y_test, pred_y))